In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

datagen = ImageDataGenerator(
    rotation_range=25,             # random rotations (-25 to +25 degrees)
    width_shift_range=0.2,         # horizontal shift
    height_shift_range=0.2,        # vertical shift
    shear_range=0.2,               # shearing
    zoom_range=0.2,                # zoom in/out
    brightness_range=[0.6, 1.4],   # vary brightness
    channel_shift_range=30.0,      # randomly shift RGB values
    horizontal_flip=True,          # random left-right flip
    vertical_flip=True,            # (use only if upside-down faces make sense)
    fill_mode="nearest"            # fill strategy for empty pixels
)

input_root = "dataset"
output_root = "dataset_augmented"

os.makedirs(output_root, exist_ok=True)

# Loop over each class folder
for class_name in os.listdir(input_root):
    input_dir = os.path.join(input_root, class_name)
    output_dir = os.path.join(output_root, class_name)
    os.makedirs(output_dir, exist_ok=True)

    # Skip if not a folder
    if not os.path.isdir(input_dir):
        continue

    # Loop over each image in the class folder
    for filename in os.listdir(input_dir):
        img_path = os.path.join(input_dir, filename)
        if os.path.isdir(img_path):  # skip subfolders
            continue

        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        # Apply ALL augmentations one by one
        augmentations = [
            {"theta": 25},   # rotation
            {"tx": 0.2 * x.shape[1]},  # width shift
            {"ty": 0.2 * x.shape[2]},  # height shift
            {"shear": 20},   # shear
            {"zx": 1.2, "zy": 1.2},  # zoom
            {"brightness": 0.7},  # brightness (dark)
            {"brightness": 1.3},  # brightness (bright)
            {"channel_shift_intensity": 20},  # color shift
            {"flip_horizontal": True},  # horizontal flip
            {"flip_vertical": True},    # vertical flip
        ]

        for idx, params in enumerate(augmentations):
            transformed = datagen.apply_transform(x[0], params)
            aug_img = array_to_img(transformed)
            aug_img.save(os.path.join(output_dir, f"aug_{idx}_{filename}"))

